In [1]:
import pandas as pd
import torch
import numpy as np
import re
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

c:\Users\dbhar\anaconda3\envs\e0_334_DLNLP\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [3]:
print(device)

cpu


In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
emb_model = {}
f = open('glove.6B.300d.txt', encoding='utf-8')
for line in f:
  #print(line)
  values = line.split()
  try:
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    emb_model[word.lower()] = coefs    
  except:
    pass
f.close()

In [5]:
emb_list = list(emb_model.keys())
def create_embedding_matrix(embedding_dict,dimension):  
  embedding_matrix=np.zeros((len(emb_list)+1,dimension))
  
  for idx,item in enumerate(emb_list):
    embedding_matrix[idx]=embedding_dict[item]
  return embedding_matrix

In [6]:
e = create_embedding_matrix(emb_model,300)

In [7]:
e.shape

(400001, 300)

In [8]:
emb_list.index('of')

3

In [9]:
e[3,:10]

array([-0.076947  , -0.021211  ,  0.21270999, -0.72232002, -0.13988   ,
       -0.12234   , -0.17521   ,  0.12137   , -0.070866  , -1.57210004])

In [11]:
emb_model['of'][:10]

array([-0.076947, -0.021211,  0.21271 , -0.72232 , -0.13988 , -0.12234 ,
       -0.17521 ,  0.12137 , -0.070866, -1.5721  ], dtype=float32)

In [47]:
a = -1*np.ones((len(emb_model['of']),1))
b = -1*torch.ones((len(emb_model['of']),1))
if ((torch.tensor(a)==b).sum() == 300):
    print('Truedddddd')

Truedddddd


In [14]:
BATCH_SIZE=2
SPLIT_RATIO = 0.8

In [55]:
def seq_to_sent_vec(seq):    
    doc = nlp(re.sub('<[^<]+?>', '', seq))
    eos_vec= -1*np.ones((len(emb_model['of']),1))
    sentences=[]    
    for token in doc.sents:        
        ls = [t.text for t in token if (t.is_alpha==True and t.is_punct==False and t.like_url==False)]
        print(' '.join(ls))
        for word in ls:       
            sentences.append(emb_model[word])
        sentences.append(eos_vec)#<EOS>
    return sentences

def seq_to_vec(seq):    
    doc = nlp(re.sub('<[^<]+?>', '', seq))
    ls = [t.text for t in doc if (t.is_alpha==True and t.is_punct==False and t.like_url==False)]
    #print(ls)
    #print(len(ls))
    vec=[]
    for word in ls:
        print(word)        
        try:
            vec.append(emb_list.index(word))
        except:
            pass    
    return vec

def label_str_to_num(label):    
    if(label.lower()=='positive'):
        return 1
    else:
        return 0

In [56]:
seq_to_vec("One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked")

One
of
the
other
reviewers
has
mentioned
that
after
watching
just
Oz
episode
you
be
hooked


[3, 0, 68, 17961, 31, 3042, 12, 49, 2641, 120, 1942, 81, 30, 12765]

In [59]:
from torch import nn
elayer = nn.Embedding.from_pretrained(torch.from_numpy(e))

In [65]:
v = seq_to_vec("One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked")
v = torch.IntTensor(v)
v = v.unsqueeze(dim=0)
print(v.size())
x = elayer(torch.IntTensor(v))
print(x.size())

One
of
the
other
reviewers
has
mentioned
that
after
watching
just
Oz
episode
you
be
hooked
torch.Size([1, 14])
torch.Size([1, 14, 300])


In [63]:
print(x.size())

torch.Size([14, 300])


In [5]:
data_csv_str = "Train dataset.csv"
data_df = pd.read_csv(data_csv_str)

In [6]:
print(data_df['review'][0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [7]:
s = seq_to_sent_vec(data_df['review'][0:2])
print(s)


TypeError: expected string or bytes-like object

In [ ]:
[16, 11, 24, 13, 12, 10, 17, 30, 15, 13, 22, 15, 43, 44, 22]

In [8]:
class CustomTextDataset(Dataset):
    def __init__(self, text, labels):
        self.labels = labels
        self.text = text
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        sample = {"Text": text, "Class": label}
        return sample

In [9]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for sample in batch:        
        label_list.append(label_str_to_num(sample['Class']))
        processed_text = torch.tensor(seq_to_vec(sample['Text']), dtype=torch.float32)        
        #label_list.append(sample['Class'])
        #processed_text = torch.tensor(sample['Text'], dtype=torch.float32)
        
        text_list.append(processed_text)        
        offsets.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.float32)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    #text_list = pad_sequence(text_list,batch_first=True)
    #text_list=torch.tensor(text_list, dtype=torch.double)
    #print(label_list.size(),offsets.size(),text_list.size())
    
    return label_list, text_list, offsets

In [11]:
DS = CustomTextDataset(data_df['review'], data_df['sentiment'])

In [12]:
from torch.utils.data import random_split

train_size = int(SPLIT_RATIO*len(DS))
test_size = len(DS) - train_size

train_dataset,val_dataset = random_split(DS, [train_size,test_size],generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_batch)
val_dataloader = DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=False, collate_fn=collate_batch)

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [8]:
some_list = [1, 10, 100, 9999, 99999]
t = torch.from_numpy(np.array(some_list, dtype=np.int))
print(t)

tensor([    1,    10,   100,  9999, 99999], dtype=torch.int32)


In [2]:
a={}
a['word'] = 'abc'
a['index'] = 222

In [3]:
a['word']

KeyError: 'abc'

In [ ]:
from torch import nn
class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.eos_marker = -1*torch.ones((len(emb_model['of']),1))
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.lstm2 = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)        
        self.sigmoid = nn.Sigmoid()
        self.hidden = self.init_hidden()
    
    def check_eos(self, x):
        return ((x==self.eos_marker).sum() == self.input_dim)

    def init_hidden(self):        
        return torch.zeros(self.n_layers, 1, self.hidden_dim)                            

    def forward(self, x):
        s = []
        input = []
        for i in range(0,x.size(0)):
            if(~self.check_eos(x[i])):
                _,(h,_) = self.lstm1(x)
                print(h.size())
                s.append(h)
            else:
                input = input.append(np.mean(np.array(s, dtype=np.float32),axis=0))
        in1 = torch.tensor(np.array(input,dtype=np.float32))
        _,(h,_) = self.lstm2(in1)
        out = self.sigmoid(self.fc(h))
        return out
  

In [ ]:
import time
def evaluate(model,criterion, val_loader):
    model.eval()
    total_acc, total_count = 0, 0
    total_loss = 0
    with torch.no_grad():
        for _, (label, text, offset) in enumerate(val_loader):            
            num_Samples = offset.size(0) 

            out=torch.zeros(offset.size(0),1)            
            for idx in range(0,num_Samples):           
              if(idx==num_Samples-1):
                  x = text[offset[idx]:-1]
              else:
                  x = text[offset[idx]:offset[idx+1]]
                             
              if(x.size(0) != 0):          
                  out[idx] = model(x.to(device))
              else:
                  out[idx] = 0.01
            out = out.squeeze()                    
            loss = criterion(out.to(device),label.to(device).float())       
            
            total_loss +=loss
            total_acc += (out.round() == label).sum().item()
            #print(total_acc)
            total_count += label.size(0)
    return total_acc/total_count, total_loss/total_count

def train(train_loader, val_loader=None, learn_rate=0.01, hidden_dim=256, EPOCHS=5):
    
    # Setting common hyperparameters
    input_dim = 300
    output_dim = 1
    n_layers = 1

    model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    print(model)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.BCELoss(reduction='sum')
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    print('-' * 59)
    print("Starting Training of model")
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()        
        total_loss = 0.
        total_acc = 0.
        counter = 0
        model.train()
        for _,(label, text, offset) in enumerate(train_loader):
            counter += 1
            model.zero_grad()              
            num_Samples = offset.size(0) 

            out=torch.zeros(offset.size(0),1)            
            for idx in range(0,num_Samples):           
              if(idx==num_Samples-1):
                  x = text[offset[idx]:-1]
              else:
                  x = text[offset[idx]:offset[idx+1]]
                             
              if(x.size(0) != 0):          
                  out[idx] = model(x.to(device))
              else:
                  out[idx] = 0.01

            out = out.squeeze()             
            loss = criterion(out.to(device),label.to(device).float())
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total_acc += (out.round() == label).sum().item()
            #print(out.size(),label.size())          
            #print(out)
            #print(label)
            #print(total_acc)
            
            if counter%10 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss={:10.4}:".format(epoch, counter, len(train_loader), total_loss/(counter*BATCH_SIZE)))
            

        current_time = time.clock()
        print("Epoch {}/{} Done, Average Train Loss={:10.4}, Train Accuracy={:10.4}".format(epoch, EPOCHS, total_loss/(BATCH_SIZE*len(train_loader)),total_acc/(BATCH_SIZE*len(train_loader))))

        if val_loader is not None:
          acc_val,loss_val = evaluate(model, criterion, val_loader)
          print("Epoch {}/{} Done, Average Val Loss={:10.4}, Average Val Accuracy={:10.4}".format(epoch, EPOCHS, loss_val,acc_val))

        print("Total Time Elapsed={} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
        print('-' * 59)

    print("Total Training Time={} seconds".format(str(sum(epoch_times))))

    return model

In [ ]:
Lstm_model = train(train_dataloader, val_dataloader, 0.1,256,50)

LSTMNet(
  (lstm): LSTM(300, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
-----------------------------------------------------------
Starting Training of model


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Epoch 1......Step: 10/40....... Average Loss=     1.438:
Epoch 1......Step: 20/40....... Average Loss=     1.078:
Epoch 1......Step: 30/40....... Average Loss=    0.9526:
Epoch 1......Step: 40/40....... Average Loss=    0.8942:
Epoch 1/50 Done, Average Train Loss=    0.8942, Train Accuracy=    0.5363


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Epoch 1/50 Done, Average Val Loss=     0.691, Average Val Accuracy=     0.525
Total Time Elapsed=115.277061 seconds
-----------------------------------------------------------
Epoch 2......Step: 10/40....... Average Loss=    0.6965:
Epoch 2......Step: 20/40....... Average Loss=    0.7006:
Epoch 2......Step: 30/40....... Average Loss=    0.7201:
Epoch 2......Step: 40/40....... Average Loss=    0.7274:
Epoch 2/50 Done, Average Train Loss=    0.7274, Train Accuracy=    0.5188
Epoch 2/50 Done, Average Val Loss=    0.7112, Average Val Accuracy=    0.5078
Total Time Elapsed=113.82074200000011 seconds
-----------------------------------------------------------
Epoch 3......Step: 10/40....... Average Loss=    0.7007:
Epoch 3......Step: 20/40....... Average Loss=    0.6922:
Epoch 3......Step: 30/40....... Average Loss=    0.7016:
Epoch 3......Step: 40/40....... Average Loss=     0.723:
Epoch 3/50 Done, Average Train Loss=     0.723, Train Accuracy=    0.5215
Epoch 3/50 Done, Average Val Loss=  